In [1]:
import json 
with open("/home/hungpv/projects/train_embedding/nanographrag/training_embedding/zalo_wiki_qa/train.json", "r") as f:
    train_data = json.load(f)

with open("/home/hungpv/projects/train_embedding/nanographrag/training_embedding/zalo_wiki_qa/test.json", "r") as f:
    test_data = json.load(f)

In [21]:
len(train_data)
data = [x for x in train_data if x["label"] == True]
len(data)

5738

In [3]:
len(test_data)

501

In [7]:
p_list = []
for p in test_data:
    for para in p["paragraphs"]:
        p_list.append(para["text"])


In [8]:
len(p_list)

2678

In [22]:
p_train = [x["text"] for x in train_data]
q_train = [x["question"] for x in data]
q_test = [x["question"] for x in test_data]
len(p_train)

18108

In [11]:
len(set(p_train + p_list))

17866

In [23]:
len(set(q_train+ q_test))

4896

In [2]:
# translate 
def Jaccard_Similarity(doc1, doc2): 
    
    # List the unique words in a document
    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [3]:
# translation model
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_vi2en = AutoTokenizer.from_pretrained("vinai/vinai-translate-vi2en-v2", src_lang="vi_VN")
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-vi2en-v2")
device_vi2en = torch.device("cuda:0")
model_vi2en.to(device_vi2en)


def translate_vi2en(vi_texts: str) -> str:
    input_ids = tokenizer_vi2en(vi_texts, padding=True, return_tensors="pt").to(device_vi2en)
    output_ids = model_vi2en.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    en_texts = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
    return en_texts

/home/hungpv/miniconda3/envs/embedding_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
batch_size = 10
text_data = [sample["text"] for sample in train_data]
from tqdm import tqdm 
english_translation = []
for i in tqdm(range(0,len(text_data), batch_size)):
    english_translation.extend(translate_vi2en(text_data[i: i + batch_size]))


  7%|▋         | 134/1811 [05:57<1:14:38,  2.67s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 39.39 GiB of which 71.31 MiB is free. Process 2396806 has 29.08 GiB memory in use. Including non-PyTorch memory, this process has 10.22 GiB memory in use. Of the allocated memory 9.14 GiB is allocated by PyTorch, and 608.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)